In [1]:
import json
import os
from pprint import pprint

from resources.similarity import CovidSimilarityResource
from corpus_index import load_corpus_index
from encoders.bert_encoder import BertEncoder
from encoders.simple_encoder import SimpleEncoder
import pandas as pd

In [23]:
corpus_index_folder = '../../../workspace/kaggle/covid19/data/corpus_index_bert'

In [5]:
corpus_index_fname = os.path.join(corpus_index_folder, 'bert-encoder-nmslib-768d.bin')
corpus_index = load_corpus_index(corpus_index_fname)

In [6]:
data_dir = "../../../workspace/kaggle/covid19/data"

In [7]:
sentences_df = pd.read_csv(os.path.join(data_dir, "sentences_with_metadata_no_phrases_blingfire.csv"))

In [8]:
def create_articles_metadata_mapping(sentences_df: pd.DataFrame) -> dict:
    sentence_id_to_metadata = {}
    for row_count, row in sentences_df.iterrows():
        sentence_id_to_metadata[row_count] = dict(
            paper_id=row['paper_id'],
            cord_uid=row['cord_uid'],
            source=row['source'],
            url=row['url'],
            publish_time=row['publish_time'],
            authors=row['authors'],
            section=row['section'],
            sentence=row['sentence'],
        )
    return sentence_id_to_metadata

In [9]:
sentence_id_to_metadata = create_articles_metadata_mapping(sentences_df)

In [10]:
sentence_encoder = BertEncoder()

In [11]:
from nlp import text_tokenizer
from nlp.cleaning import clean_tokenized_sentence

def clean_sentence(sentence) -> str:
    doc = text_tokenizer.tokenize_text(sentence)
    tokens = [str(token) for token in doc]
    return clean_tokenized_sentence(tokens)

In [18]:
import murmurhash
import numpy as np
import scipy.spatial

from corpus_index.base_index import Aggregation
from nlp.textrank import calc_textrank

class CovidSimilarity:
    def __init__(self, corpus_index, sentence_encoder, sentence_id_to_metadata, bigram_model=None, trigram_model=None):
        self.corpus_index = corpus_index
        self.sentence_encoder = sentence_encoder
        self.sentence_id_to_metadata = sentence_id_to_metadata
        self.bigram_model = bigram_model
        self.trigram_model = trigram_model

    def similar_k(self, input_sentences, limit=10, method='union', group_by='cosine', clean_sentences=True, threshold=0.7):
        """Find similar sentences.

        Args:
            input_sentences (str/list[str]): one or more input sentences.
            sentence_encoder  : encoder
            limit (int): limit result set size to ``limit``.
            corpus_index : type of corpus where to fetch the suggestions from
            db_session  : Database to get neighbors from
            method (str): aggregation method ('union', 'mean', 'pc1', 'pc2').
            group_by (str): distance metric to use to group the result set. Default is 'cosine'.

        Returns:
            list<dict>
        """
        res = []
        nearest = dict()

        if not clean_sentences:
            cleaned_sentences = input_sentences  # in case of BERT there is no need for pre-processing
        else:
            cleaned_sentences = [clean_sentence(sentence) for sentence in input_sentences]

            if self.bigram_model and self.trigram_model:
                tokenzied_sentences = [sentence.split(' ') for sentence in cleaned_sentences]
                sentences_with_bigrams = self.bigram_model[tokenzied_sentences]
                sentences_with_trigrams = self.trigram_model[sentences_with_bigrams]
                cleaned_sentences = [' '.join(sentence) for sentence in sentences_with_trigrams]

        embeddings = self.sentence_encoder.encode(cleaned_sentences)

        for idx, dist in self.corpus_index.knn_query_batch(embeddings, limit=limit, method=method):
            similarity = 1.0 - dist
            if idx not in nearest:
                nearest[idx] = similarity
            else:
                nearest[idx] = max(nearest[idx], similarity)

        results = []
        for idx, sim in nearest.items():
            if not threshold or (threshold and sim >= 0.7):
                results.append(dict(similarity=sim, metadata=self.sentence_id_to_metadata[idx], id=idx))

        return {
            "results": results,
            "sentencs": cleaned_sentences
        }

In [19]:
covid_similarity = CovidSimilarity(corpus_index, sentence_encoder, sentence_id_to_metadata)

In [20]:
def find_similar_sentences(sentences, method="union", limit=10):
    return covid_similarity.similar_k(sentences, method=method, limit=limit, clean_sentences=False, threshold=0.7)

In [41]:
find_similar_sentences(["sars-covid-2 originated from bats"])

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.11it/s]


{'results': [{'similarity': 0.7755008935928345,
   'metadata': {'paper_id': 'c630ebcdf30652f0422c3ec12a00b50241dc9bd9',
    'cord_uid': 'qzbqxjdi',
    'source': 'CZI',
    'url': 'https://doi.org/10.1007/s00134-020-05985-9',
    'publish_time': '2020',
    'authors': 'Haibo Zhang, Josef M Penninger, Yimin Li, Nanshan Zhong, Arthur S Slutsky',
    'section': 'body',
    'sentence': 'A phylogenetic analysis [3, 4] found a bat origin for the SARS-CoV-2.'},
   'id': 67275},
  {'similarity': 0.7687021493911743,
   'metadata': {'paper_id': '628224850a2206055fd6792b865f83674e3d3c29',
    'cord_uid': '1a8uevk8',
    'source': 'Elsevier',
    'url': 'https://doi.org/10.1016/j.ijantimicag.2020.105948',
    'publish_time': '2020-03-19',
    'authors': 'Li-Sheng Wang, Yi-Ru Wang, Da-Wei Ye, Qing-Quan Liu',
    'section': 'body',
    'sentence': 'The most likely source of SARS-CoV-2 is bats.'},
   'id': 126111},
  {'similarity': 0.7329314947128296,
   'metadata': {'paper_id': 'e3b40cc8e0e137c416b4

In [40]:
for task_id in [1, 2, 3, 4, 5, 6, 8, 9, 10]:
    with open(f'../answers/{task_id}.json', 'r') as in_fp:
        seed_sentences_json = json.load(in_fp)

    print(seed_sentences_json["taskName"])

    answers_results = []
    for sub_task_json in seed_sentences_json["subTasks"]:
        sub_task_description = sub_task_json["description"]
        print(f"Working on task: {sub_task_description}")
        best_sentences = sub_task_json["bestSentences"]
        relevant_sentences = covid_similarity.similar_k(best_sentences, limit=5 * len(best_sentences), method="union", clean_sentences=False, threshold=0.7)

        results = []
        for relevant_sentence in relevant_sentences["results"]:
            metadata_json = relevant_sentence["metadata"]
            if "authors" in metadata_json:
                del metadata_json["authors"]
            results.append(metadata_json)
        answers_results.append(dict(sub_task_description=sub_task_description,
                                    seed_sentences=best_sentences,
                                    relevant_sentences=results))

    with open(f'../answers/{task_id}_relevant_sentences.json', 'w+') as out_fp:
        result = dict(taskName=seed_sentences_json["taskName"], subTasks=answers_results)
        json.dump(result, out_fp)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

What is known about transmission, incubation, and environmental stability?
Working on task: Range of incubation periods for the disease in humans (and how this varies across age and health status)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Working on task: How long individuals are contagious, even after recovery.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Working on task: Measures to reach marginalized and disadvantaged populations: Data systems and research priorities and agendas incorporate attention to the needs and circumstances of disadvantaged populations and underrepresented minorities.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Working on task: Mitigating threats to incarcerated people from COVID-19, assuring access to information, prevention, diagnosis, and treatment.
Working on task: Understanding coverage policies (barriers and opportunities) related to testing, treatment, and care


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.94it/s]
